In [14]:
from transformers import AutoTokenizer, AutoModel

model_name = "allegro/herbert-base-cased"
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

text = 'Bardzo lubię lody malinowe z bitą śmietaną.'

token_ids = tokenizer(text, return_tensors='pt')['input_ids'][0]

print ([tokenizer.decode(idx) for idx in token_ids])



Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['<s>', 'Bardzo', 'lubię', 'lody', 'mali', 'nowe', 'z', 'bi', 'tą', 'śmietan', 'ą', '.', '</s>']


In [15]:
import random
lines = open('reviews_for_task3.txt').readlines()

def representation(L):
    txt = ' '.join(L)
    input_ids = tokenizer(txt, return_tensors='pt')['input_ids'].to(device)
    output = model(input_ids=input_ids)
    return output.last_hidden_state.detach().cpu().numpy()[0,0,:]

def spoil(L):
    res = []
    for w in L:
        if random.random() < 0.85:
            res.append(w)
        else:
            res.append(w.upper())
    return res
    
# random.seed(42)
random.shuffle(lines)

N = len(lines)
test_size = N // 4
train_size = N - test_size

train_lines = lines[:train_size]
test_lines = lines[train_size:]

X_train = []
y_train = []
X_test = []
y_test = []

for line in train_lines:
    L = line.split()
    y = 0 if L[0] == 'BAD' else 1
    
    x = representation(L[1:])
    y_train.append(y)
    X_train.append(x)
    
    for i in range(3):
        x = representation(spoil(L[1:]))
        y_train.append(y)
        X_train.append(x)
    
    if len(X_train) % 100 == 0:
        print (len(X_train))

for line in test_lines:
    L = line.split()
    y = 0 if L[0] == 'BAD' else 1
    
    x = representation(L[1:])
    y_test.append(y)
    X_test.append(x)
        
    if len(X_test) % 100 == 0:
        print (len(X_test))

100
200
300
400
500
600
700
800
900
1000
1100
1200
100


In [16]:
N = len(lines)
test_size = N // 4
train_size = N - test_size




In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm 

clf = LogisticRegression(random_state=0).fit(X_train, y_train)

print ('Train accuracy:', clf.score(X_train, y_train))
print ('Test accuracy:', clf.score(X_test, y_test))

#Train accuracy: 0.9348939283101683
#Test accuracy: 0.8715697036223929

Train accuracy: 0.9975
Test accuracy: 0.78


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
